Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Adding the models from assignment 2, then modifying for the L2 loss

We will need to add another variable to be variable to optimise

In [12]:
#First the not deep gradient decent graph/model
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  betal2 = tf.placeholder(tf.float32)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + betal2 * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, betal2 : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.607412
Minibatch accuracy: 7.8%
Validation accuracy: 9.9%
Minibatch loss at step 500: 2.486042
Minibatch accuracy: 76.6%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 1.701233
Minibatch accuracy: 78.9%
Validation accuracy: 78.2%
Minibatch loss at step 1500: 0.974241
Minibatch accuracy: 83.6%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 0.822908
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 0.826522
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 0.750114
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Test accuracy: 89.0%


As a comparison with the assignment 2 values, the max this time were approx82% for valid and 89% with the test.
In the previous assignment the max values were approx 75% valid and 82% test

Now for the deep gradient decent model

In [18]:
#Now to try it on the deeper network with the 1024 hidden nodes

#First initialise a variable specifying the number of hidden nodes
num_hidden_nodes = 1024

#Now reuse the code from SGD above

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  betal2 = tf.placeholder(tf.float32)
  
  # Variables.
  # Note - Now that there is a RELU function added in there is an additional /
  # weight and bias required before the logits output
  weight1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  weight2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  bias1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  bias2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # Instead of matmul straight to logits, the matmul is put in RELU as layer1 /
  # the layer 1 is then matmul with the new weight and bias to the logits
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weight1) + bias1)
  logits = tf.matmul(layer1, weight2) + bias2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + betal2 * tf.nn.l2_loss(weight1) + betal2 * tf.nn.l2_loss(weight2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # We also need to add the extra steps into the valid and test checks
  train_prediction = tf.nn.softmax(logits)
  layer1_valid = tf.nn.relu(
    tf.matmul(tf_valid_dataset, weight1) + bias1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(layer1_valid, weight2) + bias2)
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1)
  test_prediction = tf.nn.softmax(tf.matmul(layer1_test, weight2) + bias2)

In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, betal2 : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 613.054443
Minibatch accuracy: 8.6%
Validation accuracy: 24.4%
Minibatch loss at step 500: 198.118179
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 115.240082
Minibatch accuracy: 84.4%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 68.606529
Minibatch accuracy: 88.3%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 41.306290
Minibatch accuracy: 87.5%
Validation accuracy: 85.2%
Minibatch loss at step 2500: 25.180372
Minibatch accuracy: 89.1%
Validation accuracy: 85.9%
Minibatch loss at step 3000: 15.509544
Minibatch accuracy: 86.7%
Validation accuracy: 87.0%
Test accuracy: 93.2%


As a comparison from assignment 2. Values were valid=82% and test=89%. Values are now valid=87% and test=93.2%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
#Start with the same model as previous, but limit the batch number.

#First initialise a variable specifying the number of hidden nodes
num_hidden_nodes = 1024

#Now reuse the code from SGD above

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  betal2 = tf.placeholder(tf.float32)
  
  # Variables.
  # Note - Now that there is a RELU function added in there is an additional /
  # weight and bias required before the logits output
  weight1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  weight2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  bias1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  bias2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # Instead of matmul straight to logits, the matmul is put in RELU as layer1 /
  # the layer 1 is then matmul with the new weight and bias to the logits
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weight1) + bias1)
  logits = tf.matmul(layer1, weight2) + bias2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + betal2 * tf.nn.l2_loss(weight1) + betal2 * tf.nn.l2_loss(weight2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # We also need to add the extra steps into the valid and test checks
  train_prediction = tf.nn.softmax(logits)
  layer1_valid = tf.nn.relu(
    tf.matmul(tf_valid_dataset, weight1) + bias1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(layer1_valid, weight2) + bias2)
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1)
  test_prediction = tf.nn.softmax(tf.matmul(layer1_test, weight2) + bias2)

In [12]:
num_steps = 3001

#Limit the number of batches to 5. So training data is only 5*128 big
number_of_batches = 5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #Restrict the offset to 5 different 128 batch size chunks
    offset = (step % number_of_batches) * batch_size
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, betal2 : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 630.518677
Minibatch accuracy: 14.1%
Validation accuracy: 19.3%
Minibatch loss at step 500: 190.330078
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 115.426743
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 1500: 70.001221
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 2000: 42.452763
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 2500: 25.746204
Minibatch accuracy: 100.0%
Validation accuracy: 75.6%
Minibatch loss at step 3000: 15.615737
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Test accuracy: 83.3%


By inspection, limiting the training data to only 5 blocks meant the training data was quickly got to 100% accuracy,
however the valid data plateaued and the test accuracy was much lower.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [13]:
#First initialise a variable specifying the number of hidden nodes
num_hidden_nodes = 1024

#Now reuse the code from SGD above

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  betal2 = tf.placeholder(tf.float32)
  #Set up the keep probability (dropout) as a variable so it can be set for training and testing
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  # Note - Now that there is a RELU function added in there is an additional /
  # weight and bias required before the logits output
  weight1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  weight2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  bias1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  bias2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # Instead of matmul straight to logits, the matmul is put in RELU as layer1 /
  # the layer 1 is then matmul with the new weight and bias to the logits
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weight1) + bias1)
  layer1_drop =  tf.nn.dropout(layer1, keep_prob)
  logits = tf.matmul(layer1_drop, weight2) + bias2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + betal2 * tf.nn.l2_loss(weight1) + betal2 * tf.nn.l2_loss(weight2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # We also need to add the extra steps into the valid and test checks
  train_prediction = tf.nn.softmax(logits)
  layer1_valid = tf.nn.relu(
    tf.matmul(tf_valid_dataset, weight1) + bias1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(layer1_valid, weight2) + bias2)
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1)
  test_prediction = tf.nn.softmax(tf.matmul(layer1_test, weight2) + bias2)

In [16]:
#Start with trying 50% dropout on training and no dropout on valid and test

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, betal2 : 1e-3, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict = {
                    tf_train_dataset : batch_data, tf_train_labels : batch_labels, betal2 : 1e-3, keep_prob : 1}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict = {
                tf_train_dataset : batch_data, tf_train_labels : batch_labels, betal2 : 1e-3, keep_prob : 1}), test_labels))

Initialized
Minibatch loss at step 0: 787.277344
Minibatch accuracy: 12.5%
Validation accuracy: 22.0%
Minibatch loss at step 500: 204.380066
Minibatch accuracy: 78.9%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 116.171837
Minibatch accuracy: 78.1%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 69.225555
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 41.495430
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 25.285210
Minibatch accuracy: 82.8%
Validation accuracy: 84.6%
Minibatch loss at step 3000: 15.476610
Minibatch accuracy: 82.0%
Validation accuracy: 85.0%
Test accuracy: 91.9%


Tried with 80 keep prob and got 86% valid and 92.6% test. With 50% get 85% valid and 91.9% test.
will try 90% keep prob.

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, betal2 : 1e-3, keep_prob : 0.9}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict = {
                    tf_train_dataset : batch_data, tf_train_labels : batch_labels, betal2 : 1e-3, keep_prob : 1}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict = {
                tf_train_dataset : batch_data, tf_train_labels : batch_labels, betal2 : 1e-3, keep_prob : 1}), test_labels))

Initialized
Minibatch loss at step 0: 806.083862
Minibatch accuracy: 10.2%
Validation accuracy: 26.9%
Minibatch loss at step 500: 201.595123
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 115.806343
Minibatch accuracy: 78.1%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 68.499649
Minibatch accuracy: 86.7%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 41.220131
Minibatch accuracy: 89.1%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.173098
Minibatch accuracy: 86.7%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 15.469470
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Test accuracy: 92.8%


From testing,it appears the model is more accurate without the dropout in this configuration

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [65]:
#First initialise a variable specifying the number of hidden nodes

#For this test/try at optimising I will try adding an additional layer and apply the new learning rate
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 512

batch_size = 256

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #Set up the keep probability (dropout) as a variable so it can be set for training and testing
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  # Note - Now that there is a RELU function added in there is an additional /
  # weight and bias required before the logits output
  weight1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes1], stddev=np.sqrt(2.0 / (image_size * image_size))))
  weight2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
  weight3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
  bias1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  bias2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  bias3 = tf.Variable(tf.zeros([num_labels]))
  global_step = tf.Variable(0)  # count the number of steps taken.
  
  # Training computation.
  # Instead of matmul straight to logits, the matmul is put in RELU as layer1 /
  # the layer 1 is then matmul with the new weight and bias to the logits
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weight1) + bias1)
  layer1_drop =  tf.nn.dropout(layer1, keep_prob)
  layer2 = tf.nn.relu(tf.matmul(layer1_drop, weight2) + bias2)
  layer2_drop =  tf.nn.dropout(layer2, keep_prob)
  logits = tf.matmul(layer2_drop, weight3) + bias3
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 4000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  # We also need to add the extra steps into the valid and test checks
  train_prediction = tf.nn.softmax(logits)
  layer1_valid = tf.nn.relu(
    tf.matmul(tf_valid_dataset, weight1) + bias1)
  layer2_valid = tf.nn.relu(
    tf.matmul(layer1_valid, weight2) + bias2)                                                                                 
  valid_prediction = tf.nn.softmax(
    tf.matmul(layer2_valid, weight3) + bias3)
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1)
  layer2_test = tf.nn.relu(tf.matmul(layer1_test, weight2) + bias2)
  test_prediction = tf.nn.softmax(tf.matmul(layer2_test, weight3) + bias3)

In [69]:
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict = {
                    tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict = {
                tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1}), test_labels))

Initialized
Minibatch loss at step 0: 2.517720
Minibatch accuracy: 9.4%
Validation accuracy: 24.0%
Minibatch loss at step 1000: 0.337759
Minibatch accuracy: 89.1%
Validation accuracy: 88.1%
Minibatch loss at step 2000: 0.293609
Minibatch accuracy: 91.8%
Validation accuracy: 89.3%
Minibatch loss at step 3000: 0.329219
Minibatch accuracy: 89.5%
Validation accuracy: 89.8%
Minibatch loss at step 4000: 0.237661
Minibatch accuracy: 91.8%
Validation accuracy: 90.2%
Minibatch loss at step 5000: 0.319058
Minibatch accuracy: 91.0%
Validation accuracy: 90.5%
Minibatch loss at step 6000: 0.185879
Minibatch accuracy: 92.6%
Validation accuracy: 90.7%
Minibatch loss at step 7000: 0.172223
Minibatch accuracy: 94.1%
Validation accuracy: 90.9%
Minibatch loss at step 8000: 0.206290
Minibatch accuracy: 92.6%
Validation accuracy: 90.6%
Test accuracy: 96.0%


Simply adding an additional layer, increasing the batch size and number of steps has increased the accuracy significantly

Now try droppout of 0.5

In [72]:
#First initialise a variable specifying the number of hidden nodes

#For this test/try at optimising I will try adding an additional layer and apply the new learning rate
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 512

batch_size = 256

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #Set up the keep probability (dropout) as a variable so it can be set for training and testing
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  # Note - Now that there is a RELU function added in there is an additional /
  # weight and bias required before the logits output
  weight1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes1], stddev=np.sqrt(2.0 / (image_size * image_size))))
  weight2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
  weight3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
  bias1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  bias2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  bias3 = tf.Variable(tf.zeros([num_labels]))
  global_step = tf.Variable(0)  # count the number of steps taken.
  
  # Training computation.
  # Instead of matmul straight to logits, the matmul is put in RELU as layer1 /
  # the layer 1 is then matmul with the new weight and bias to the logits
  layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weight1) + bias1)
  layer1_drop =  tf.nn.dropout(layer1, keep_prob)
  layer2 = tf.nn.relu(tf.matmul(layer1_drop, weight2) + bias2)
  layer2_drop =  tf.nn.dropout(layer2, keep_prob)
  logits = tf.matmul(layer2_drop, weight3) + bias3
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 

  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 4000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  # We also need to add the extra steps into the valid and test checks
  train_prediction = tf.nn.softmax(logits)
  layer1_valid = tf.nn.relu(
    tf.matmul(tf_valid_dataset, weight1) + bias1)
  layer2_valid = tf.nn.relu(
    tf.matmul(layer1_valid, weight2) + bias2)                                                                                 
  valid_prediction = tf.nn.softmax(
    tf.matmul(layer2_valid, weight3) + bias3)
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1)
  layer2_test = tf.nn.relu(tf.matmul(layer1_test, weight2) + bias2)
  test_prediction = tf.nn.softmax(tf.matmul(layer2_test, weight3) + bias3)

In [74]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict = {
                    tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict = {
                tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1}), test_labels))

Initialized
Minibatch loss at step 0: 2.610833
Minibatch accuracy: 10.9%
Validation accuracy: 34.7%
Minibatch loss at step 1000: 0.449385
Minibatch accuracy: 84.4%
Validation accuracy: 86.9%
Minibatch loss at step 2000: 0.402917
Minibatch accuracy: 88.7%
Validation accuracy: 87.8%
Minibatch loss at step 3000: 0.431737
Minibatch accuracy: 87.9%
Validation accuracy: 89.0%
Minibatch loss at step 4000: 0.336464
Minibatch accuracy: 89.8%
Validation accuracy: 89.2%
Minibatch loss at step 5000: 0.476275
Minibatch accuracy: 87.5%
Validation accuracy: 89.4%
Minibatch loss at step 6000: 0.299556
Minibatch accuracy: 90.2%
Validation accuracy: 89.7%
Minibatch loss at step 7000: 0.263837
Minibatch accuracy: 91.0%
Validation accuracy: 89.9%
Minibatch loss at step 8000: 0.317693
Minibatch accuracy: 90.6%
Validation accuracy: 90.1%
Minibatch loss at step 9000: 0.350384
Minibatch accuracy: 90.2%
Validation accuracy: 90.4%
Test accuracy: 95.9%
